In [1]:
import os, csv, time
import pandas as pd
import sparql
from collections import Counter
os.getcwd()

'/Users/Tristan/books/src'

In [2]:
# local modules
import config
import sparql


First we create an empty CSV file with the colums to fill.

## Find additional info

setup the data from the csv file to a pd dataframe and set the index of the dataframe to the titles of the books. The latter will be useful later.

In [3]:
filename = config.dataset_dir + '/output/bookdata_alt.csv'
info = pd.read_csv(filename)
info.index = info['title']


In [4]:
keys, _ = sparql.book_info()

In [5]:
def books_info(title_list=[], verbose=False):
    
    print('searching %i titles' % len(title_list))
    data_per_book = []
    unknown_titles = []
    # keys, _ = sparql.book_info()
    for title in title_list:
        # TODO try a couple of times
        # book info returns     
        #  ('ok', (keys, [info_dict]) ) | ('error', 'http_error')
        (success, result) = sparql.book_info(title)
        if verbose: print(success, len(result[1]))
        if success == 'ok':
            _, results = result
            if not results == []:
                # ignore all but the first result
                # TODO save results that have an author
                data_per_book.append(results[0])
            else: # 'http_error'
                if verbose: print('error', title)
                unknown_titles.append(title)
        else:
            unknown_titles.append(title)
        
 
    return data_per_book, unknown_titles



Run the function books_info to get the data for the books found in dbpedia and the books that where not found in dbpedia

In [6]:
v = False
results, unknown_titles = books_info(info['title'], verbose=v)

print('n results',len(results))
print('n failed (not found)',len(unknown_titles))


searching 698 titles
n results 285
n failed (not found) 413


Delete rows from the dataframe where the book is not found in dbpedia. Without this data, the row is useless.

In [7]:
for title in unknown_titles:
    info = info.drop(title)
    


Next, we want to find the genres for all books

In [8]:
def find_genres(dic):
    to_strip = 'http://dbpedia.org/resource/'
    genre_list = []
    title_list = []
   
    for result in dic:
        try:
            genre = (result['genre']['value'])
            genre = genre.replace(to_strip, '')
            genre = genre.replace('_', ' ')
            genre_list.append(genre)
            
        except KeyError as error:
            genre = 'unknown'
            genre_list.append(genre)
            # Output expected KeyErrors.
            continue
            
  
    # COMMENTED CODE DOES NOT WORK YET  
    genres = pd.Series(genre_list)
    info['genre'] = genres.values
    unique_genres = set(genre_list)
    genre_distribution = Counter(genre_list)
    print(len([genre for genre in genres if genre != "unknown"]))
    return unique_genres, genre_distribution, info

unique_genres, genre_distribution, info = find_genres(results)
print(genre_distribution)
info = info[info.genre != 'unknown']

info.to_csv(config.dataset_dir + '/output/final_data.csv')

152
Counter({'unknown': 133, "Children's literature": 17, 'Novel': 15, 'Adventure novel': 11, 'Fantasy novel': 8, 'Science fiction': 6, 'Historical novel': 6, 'Novella': 6, 'Short story': 5, 'Fiction': 3, 'War novel': 3, 'Bildungsroman': 3, 'Thriller (genre)': 3, 'Essay': 3, 'Short stories': 3, "Children's novel": 2, 'Horror fiction': 2, 'Bush poetry': 2, 'Adventure fiction': 2, 'Gothic fiction': 2, 'Historical fiction': 2, 'Autobiographical novel': 2, 'Science fantasy': 2, 'Lost World (genre)': 2, 'Detective fiction': 2, "Children's book": 2, 'Christian mythology': 2, 'Comedy novel': 2, 'Crime fiction': 1, 'History': 1, 'Horror novel': 1, 'Spiritual autobiography': 1, 'Travel literature': 1, 'Young adult novel': 1, 'Nonsense poetry': 1, 'Non-Fiction': 1, "Children's fiction": 1, 'Fantasy': 1, 'Theodicy': 1, 'Science Fantasy': 1, 'Philosophical fiction': 1, 'Autobiography': 1, 'Gothic novel': 1, 'Romance novel': 1, 'Non-fiction': 1, 'Planetary romance': 1, 'Novel of manners': 1, 'Short

In [9]:
print(unique_genres)

{'Fairy tale', 'Historical Novel', 'Bush poetry', 'Science Fantasy', 'Thriller (genre)', 'Utopian novel', 'Romance novel', 'Nonsense poetry', "Children's fiction", 'Bildungsroman', 'Gothic fiction', 'Literary realism', 'Gothic novel', 'Fantasy', 'Biographical novel', 'Spiritual autobiography', 'Short story cycle', 'Novel', 'Ruritanian romance', 'Detective fiction', 'War novel', 'Horror novel', 'Essay', 'Adventure novel', 'Mystery novel', 'Nature writing', 'Non-Fiction', 'Historical novel', 'Fiction', 'Short stories', 'Adventure fiction', 'Science fiction', 'Short story', 'unknown', 'Poetry', 'Comedy novel', 'Autobiography', "Children's literature", 'Juvenile literature', 'Science fantasy', 'Novel of manners', 'Autobiographical novel', 'Christian mythology', 'Satire', 'Fantasy novel', "Children's book", 'Christian Apologetics', 'Theodicy', 'Chivalric romance', 'Travel literature', 'Planetary romance', 'Photography', "Children's novel", 'Historical fiction', 'Lost World (genre)', 'Philos

In [10]:
sanitized = [sparql.sanitize(t) for t in unknown_titles]
results, unknown_titles = books_info(sanitized, verbose=v)
print('n results',len(results))
print('n failed (not found)',len(unknown_titles))

searching 413 titles
n results 0
n failed (not found) 413


In [11]:
# TODO string.strip
string = "sdf ' df"
string = string.strip('"`')
string = string.strip("'")
string

"sdf ' df"

In [12]:
info['title'][0:10]

title
The Wind in the Willows              The Wind in the Willows
The Amateur Cracksman                  The Amateur Cracksman
A Child's History of England    A Child's History of England
Heretics                                            Heretics
The Golden Road                              The Golden Road
The White People                            The White People
The Castle of Otranto                  The Castle of Otranto
Bride of Lammermoor                      Bride of Lammermoor
St. Ives                                            St. Ives
Allan Quatermain                            Allan Quatermain
Name: title, dtype: object